In [1]:
import torch
import sigkernel2
import sigkernel
import csv
import scipy
import timeit
import math
device = torch.cuda.device('cuda')

In [2]:
def generate(batch_size, length, dimension, device = torch.device('cpu')):
  random_walks = torch.randn(batch_size, length, dimension, dtype = torch.double, device = device) / math.sqrt(length)
  start = torch.zeros([batch_size, 1, dimension], device=device, dtype=torch.double)
  random_walks = torch.cat((start, random_walks), dim=1)
  random_walks = torch.cumsum(random_walks, dim=1)
  return random_walks

def median_distance(X, Y):
    A = X.shape[0]
    M = X.shape[1]
    N = Y.shape[1]
    Xs = torch.sum(X**2, dim=2)
    Ys = torch.sum(Y**2, dim=2)
    dist = -2.*torch.bmm(X, Y.permute(0,2,1))
    dist += torch.reshape(Xs,(A,M,1)) + torch.reshape(Ys,(A,1,N))
    return dist.view(A, -1).median()
  
def tensor_norm(x, C=2, alpha=1):
  return C - (C - 1) / (x.sqrt())**alpha

def guess(norm, value):
  if norm > value:
    return math.sqrt((-1.0 - math.sqrt(1 - 2 * (1 - value))) / (-1.0 - math.sqrt(1 - 2 * (1 - norm))))
  return math.sqrt((-1.0 + math.sqrt(1 - 2 * (1 - value))) / (-1.0 + math.sqrt(1 - 2 * (1 - norm))))
def guess2(norm, value):
  return math.sqrt((-1.0 + math.sqrt(1 - 2 * (1 - value))) / (-1.0 + math.sqrt(1 - 2 * (1 - norm))))

def guess3(norm, value):
  return (value - 1) / (norm - 1)

In [11]:
X = generate(30, 50, 10, device = torch.device('cuda:0'))

In [12]:
sig_x2 = sigkernel2.RobustSigKernel(sigkernel2.RBFKernel(median_distance(X, X).cpu().item()), 2, normalizer=lambda x : tensor_norm(x, C=2, alpha=0.5))
#sig_x2 = sigkernel2.RobustSigKernel(sigkernel2.LinearKernel(1), 2, normalizer=lambda x : tensor_norm(x, C=2, alpha=1))
sig_x = sigkernel2.SigKernel2(sigkernel2.RBFKernel(median_distance(X, X).cpu().item()), 2)
#sig_x = sigkernel2.SigKernel2(sigkernel2.LinearKernel(1), 2)

In [13]:
sig_x2.kernel(X, X, maxit=500, rel_tol=1e-8, abs_tol=1e-5)

/home/paperspace/.local/lib/python3.9/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 30 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


tensor([1.6752, 1.6229, 1.5872, 1.6427, 1.6602, 1.6656, 1.6082, 1.7063, 1.5798,
        1.6946, 1.6591, 1.6262, 1.6292, 1.6579, 1.6698, 1.6856, 1.5712, 1.6787,
        1.6849, 1.6039, 1.7270, 1.6767, 1.6628, 1.6546, 1.6870, 1.6178, 1.6126,
        1.7310, 1.5395, 1.6658], device='cuda:0', dtype=torch.float64)

In [118]:
Y = X[(test > 10).nonzero()[1]]

In [119]:
norms = sig_x.kernel(Y, Y)
print(norms)
print(guess(norms.cpu().item(), tensor_norm(norms, alpha=0.8).cpu().item()))
print(guess2(norms.cpu().item(), tensor_norm(norms, alpha=0.8).cpu().item()))
print(guess3(norms.cpu().item(), tensor_norm(norms, alpha=0.8).cpu().item()))

tensor([79.8611], device='cuda:0', dtype=torch.float64)
0.43968163600917154
0.2328500380033638
0.010481640986550021


In [121]:
lambdas = torch.tensor([0.43968163600917154], dtype=torch.float64, device=X.device)
#sig_x2.norms(Y, maxit=110, rel_tol=1e-8, abs_tol=1e-5)
sig_x.robust_kernel(Y, norms, lambdas, n=200, rel_tol=1e-8, abs_tol=1e-5)

(tensor([6.2308e+179], device='cuda:0', dtype=torch.float64),
 tensor([-1.3047e+230], device='cuda:0', dtype=torch.float64),
 tensor([13.5300], device='cuda:0', dtype=torch.float64))

In [122]:
sig_x2.kernel(Y, Y, maxit=10000, rel_tol=1e-8, abs_tol=1e-5)

tensor([1.8267], device='cuda:0', dtype=torch.float64)

In [123]:
sig_x2.guess_norm(norms, sig_x2.normalizer(norms))

tensor([0.2329], device='cuda:0', dtype=torch.float64)